In [0]:

from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket, specified_bucket
from aaintdatapipeline.core.fs.device import unified_bucket
import zlib


#s3 = S3Bucket(Conf(bucket_name='prod_appannie_ios'))


#spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-03-21/android/sbe_est_app/10/", sep="\t").distinct('product_id').count()

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-est-category-rank.v1/fact/granularity=daily/date=2019-03-21/device_code=android-all/").count()


#s3 = S3Bucket(Conf(bucket_name='b2b-prod-int-data-pipeline-unified'))
#file_list = s3.list("unified/app-tech.store.app-rank.v1/fact/")
s3 = S3Bucket(Conf(bucket_name = 'prod_appannie_ios'))
file_list = s3.list('country-ranks/2015-12-11/23/143441/')

conf = Conf(
        bucket_name='prod_appannie_ios',
        bucket_class=S3Bucket
    )
path = "country-ranks/{_date}/23/{_country_id}".format(_date='2015-12-11', _country_id=143441)
bucket = specified_bucket(conf)
raw_ios_df= []
for key in bucket.list(path):
    print key
    if 'md5' in key or 'SUCCESS' in key:
        continue
    raw_data = zlib.decompress(bucket.get(key))
    #print raw_data
    for _data in raw_data.splitlines():
        data = _data.split("\t")
        if int(data[3]) in (200, 201, 202):
            continue
        if len(data) == 5:
            data_dic = {
                'date': data[0],
                'country_id': data[1],
                'category_id': data[2],
                'feed_id': data[3],
                'rank': data[4].split(" ")
            }
            raw_ios_df.append(data_dic)
print raw_ios_df


#spark.read.parquet("s3://prod_appannie_ios/country-ranks/2019-02-11/23/143441/").show(1)


In [0]:

from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket, specified_bucket
from aaintdatapipeline.core.fs.device import unified_bucket
import zlib


s3 = S3Bucket(Conf(bucket_name = 'prod_appannie_android'))
file_list = s3.list('country-ranks/2018-12-11/23/10/')

conf = Conf(
        bucket_name='prod_appannie_android',
        bucket_class=S3Bucket
    )
path = "country-ranks/{_date}/23/{_country_id}".format(_date='2019-02-03', _country_id='US')
bucket = specified_bucket(conf)
raw_gp_df = []
for key in bucket.list(path):
    if 'md5' in key or 'SUCCESS' in key:
        continue
    raw_data = zlib.decompress(bucket.get(key))
    for _data in raw_data.splitlines():
        data = _data.split(",")
        if len(data) == 5:
            rank_class_list = data[4].split(" ")
            rank_class_list = [name.split("-")[1] for name in rank_class_list]
            data_dic = {
                'date': data[0],
                'country_code': data[1],
                'category_name': data[2],
                'feed_id': data[3],
                'rank': rank_class_list
            }
            raw_gp_df.append(data_dic)
print raw_gp_df[0]



#spark.read.parquet("s3://prod_appannie_ios/country-ranks/2019-02-11/23/143441/").show(1)


In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import get_random_ios_feed_id, get_random_date, get_unified_data_path, get_ios_raw_df
    
raw_data = get_ios_raw_df('2015-12-11', 'US')
print raw_data[71]

In [0]:

start_date = '2013-02-28'
end_date = '2019-06-02'

r = []
start = string_to_datetime(start_date)
delta = get_total_days(start_date, end_date)
for i in range(delta):
    r.append(datetime_to_string(start + timedelta(days=i)))
return r

s3 = S3Bucket(Conf(bucket_name='b2b-prod-int-data-pipeline-unified'))
result = []
missed = []
unified_count = s3.list("unified/app-tech.store.app-rank.v1/fact/")
for item in unified_count:
    result.append(item.split('/')[3].split('=')[1:])
for date in r:
    if date not in result:
        print 'android_all', missed.append(date)

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-03-21/android/sbe_est_app/10/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-est-category-rank.v1/fact/granularity=daily/date=2019-03-21/device_code=android-all/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2019-02-11/
#aws s3 ls s3://prod_appannie_ios/country-ranks/2019-02-11/23/143441/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2017-12-28/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/date=2019-06-25/device_code=android-all/
#aws s3 ls s3://b2c-prod-int-data-pipeline-unified-paid-aso/unified/app-tech.market.aso-keyword-est.v1/fact/granularity=monthly/
#aws s3 ls s3://prod.appannie.mktint.data/market_pubapp_adapp_visibility_fact/granularity=DAY/date=2019-05-19/country_code=AT/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative-fact.v1/fact/granularity=weekly/date=2019-01-19/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp-pubapp-visibility.v1/fact/date=2019-06-18/
#aws s3 ls s3://prod.appannie.mktint.data/
#aws s3 ls s3://b2c-prod-advanced-review/oss/ADVANCED_REVIEW_TERM_SCORE/version=1.0.0/language=en/process_date=2019-07-06/
#aws s3 ls s3://prod.appannie.mktint.data/market_fact/granularity=MONTH/date=2019-05-31/country_code=HK/
aws s3 ls s3://prod.appannie.mktint.data/oss/TRAFFIC_SHARE/routine/granularity=daily/

In [0]:


#spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=MONTH/date=2019-05-31/")
#spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=MONTH/date=2019-05-31/country_code=HK/").limit(1).filter('creative_id is null').collect()

spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVANCED_REVIEW_TERM_SCORE/version=1.0.0/language=en/process_date=2019-07-06/").limit(1).show()


In [0]:

import datetime
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
        
#daily = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.domain_basic_m where date = '2019-03-31'$$)t(cnt bigint);;"""
#daily = """select max(date) from plproxy.execute_select($$select max(date) as cnt from mw.domain_x_domain_m$$)t(date date);"""
#daily = """select distinct(device_code),date,country_code from plproxy.execute_select($$select distinct(device_code), date,country_code from mw.domain_basic_w where country_code='us'$$)t(device_code char, date date,country_code char);"""
#daily = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.domain_browser_w$$)t(cnt bigint);;"""
#daily = """select distinct(device_code) from plproxy.execute_select($$select distinct(device_code) from mw.domain_x_app_m where date = '2019-03-31'$$)t(device_code char);"""
#weekly_daily = """select distinct(country_code) from plproxy.execute_select($$select distinct(country_code) as cnt from mw.app_x_domain_m where date = '2019-03-31'$$)t(country_code char);;"""
#weekly_daily = """select distinct(country_code) from plproxy.execute_select($$select distinct(country_code) as cnt from mw.category_w where date = '2019-04-06'$$)t(country_code char);;"""

daily = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.app_x_domain_{granularity} where date = '2019-04-30' and device_code = '{device_code}' and country_code='us'$$)t(cnt bigint);""".format(
                granularity='m',
                device_code='ip',
                date=2019-04-30
            )
rows, _ = select(daily, None)
#weekly_rows, _ = select(weekly_daily, None)
print rows


In [0]:


#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2018-01-19/device_code=ios-phone/").filter("country_code='us',category_id='100015', app_id = '548580856'").show()

spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.market.aso-keyword-rank.v3/fact/date=2019-06-01/").filter("device_code = 'ios-phone' and keyword_id in (17703,757917,1573993,5027611,396547)").select('app_ids').collect()
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.market.aso-keyword-rank.v3/fact/date=2019-06-01/").filter("device_code = 'ios-phone'").show(1)

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.market.aso-keyword-rank.v3/fact/date=2019-06-01/").select('keyword_type').distinct().collect()
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/date=2019-06-25/device_code=android-all/").select('country_code').distinct().collect()
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app.v1/dimension").filter("category_id in (100002, 100003, 100004, 100005, 100006, 100007, 100008)").select("id").show(1)
#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app.v1/dimension").select("category_id").distinct().collect()


In [0]:


spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2018-01-19/desdfsdfsdfvice_code=ios-phone/").show()


In [0]:
%%sh

#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.ad_app_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=android-phone/country_code=AR/

#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=android-pad/country_code=AU/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_pubapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_publisher_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-phone/country_code=US/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.pub_app_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.pubapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/

#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2015-12-11/device_code=ios-tablet/
#aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2012-12-28/
aws s3 ls s3://prod_appannie_android/country-ranks/2019-02-03/23/
#aws s3 ls s3://prod_appannie_ios/country-ranks/


In [0]:

app-int.market.ad_app_category_rank.v1/
app-int.market.adapp_fact.v1/
app-int.market.adapp_pubapp_fact.v1/
app-int.market.adapp_publisher_fact.v1/
app-int.market.creative_category_rank.v1/
app-int.market.creative_fact.v1/
app-int.market.pub_app_category_rank.v1/
app-int.market.pubapp_fact.v1/

In [0]:

#spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.ad_app_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=android-phone/country_code=AR/").show(1)



df = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=android-pad/country_code=AU/")

types = [f for f in df.columns]
print types

In [0]:



try:
    unified_count = spark.read.parquet(get_unified_data_path(APP_STORE_RANK_UNIFIED_URN, date, 'sdfsf')).count()
    assert unified_count > 0, "No app store rank in unified bucket at {} on {}.".format(date, device_code)
except spark.sql.utils.AnalysisException:
    print("No app store rank data path in unified bucket at {} on {}.".format(date, device_code))

In [0]:


#df = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.ad_app_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=android-phone/country_code=AR/")
#print [f for f in df.columns]
#df = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=android-pad/country_code=AU/")
#print [f for f in df.columns]
#df = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_pubapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/")
#print [f for f in df.columns]
#df = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.adapp_publisher_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/")
#print [f for f in df.columns]
df1 = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-phone/country_code=US/")
print [f for f in df1.columns]
df2 = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.creative_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-phone/country_code=US/")
print [f for f in df2.columns]
df4 = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.pubapp_fact.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-pad/country_code=US/")
print [f for f in df4.columns]
df3 = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-int.market.pub_app_category_rank.v1/fact/granularity=weekly/date=2018-07-14/device_code=ios-phone/country_code=BR/")
print [f for f in df3.columns]





In [0]:


spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-rank.v1/fact/date=2015-12-11/device_code=ios-tablet/").filter("country_code='US'and category_id=100035 and app_id = 415318656 and revenue is not null").collect()



In [0]:
 
import datetime
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
 

In [0]:

# df = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2016-08-01/ios/sbe_est_app/143441/", sep="\t")
# print(df.count())
# print(df.select('_c5').distinct().count())

unified = spark.read.parquet("s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app-est-category-rank.v1/fact/granularity=daily/date=2016-08-01/device_code=ios-phone/")
print(unified.select('app_id').filter('free_app_download is not null and country_code = "US"').distinct().count())

raw = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2016-08-01/ios/sbe_est_app/143441/", sep="\t")
print(raw.filter('_c4 = 0').select('_c5').distinct().count())

In [0]:

from aaintdatapipeline.core.pipeline import schema, type_
from aaintdatapipeline.core.utils.identifier import virtual_id
from aaintdatapipeline.core.fs.driver.unified import (
    Query,
    UnifiedDTO,
    exist,
    read)


urn = Urn(
    identifier=virtual_id(),
    manipulation=type_.ManipulationType.INSERT,
    namespace='app-tech.store.app-est-category-rank.v1',
    data_type=type_.DataType.METRIC
)

urn.query = Query(
    wheres=[
        {
            "data_type": u"metric",
            "date": [u'2018-11-30'],
            "_identifier": 1
        }
    ]
)

unified.read(spark, urn)


In [0]:


import psycopg2
import datetime

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_citus_db",
        user="citus_bdp_usage_qa",
        host="10.2.10.132",
        password="dNzWtSV3pKTx",
        port=5432
    )
)
sql = "select * from store.store_app_rank_fact_v1 where granularity = 'hourly' and hour = 23 and country_code = 'NE' and device_code = 'android-all' limit 1;"


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
test_result = query(citus_dsn_, sql)
print test_result


In [0]:

import psycopg2

from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings

PG_AA_HOSTS = [('54.210.244.2', 5432)]
PG_AA_NAME = 'aa'
PG_AA_ACCESS_ID = 'app_tomcat'
PG_AA_SECRET_KEY = 'TMV!PYT02X*w'

urn = Urn(
    namespace='app-qa.data.validation.v1',
    owner='app-qa'
)

#sql = "SELECT tablename FROM pg_tables"
sql = "SELECT distinct(category_id) FROM app"

template = "PG_AA_ANDROID" + "_{property}"
host, port = getattr(settings, template.format(property='HOSTS'))[0]
conn = psycopg2.connect(
    database=getattr(settings, template.format(property='NAME')),
    user=getattr(settings, template.format(property='ACCESS_ID')),
    password=getattr(settings, template.format(property='SECRET_KEY')),
    host=host,
    port=port)
cur = conn.cursor()
cur.execute(sql)
res = cur.fetchall()
print res


In [0]:
%%sh

#curl -XGET int-es-kibana-v5.appannie.org:9200/_cat/indice
#curl -XGET aa-int-prod-es-v5.appannie.org:9200/aso-kpi-svi*
#curl -XGETcurl -XGET http://internal-aa-prod-int-elasticsearch-v5-1589259492.us-east-1.elb.amazonaws.com:19200/aso*/_search?pretty
curl -XGET http://internal-aa-int-uds-elasticsearch-124841367.us-east-1.elb.amazonaws.com:19200/sdk*/_search?pretty

In [0]:
 

import psycopg2
import datetime

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_mkt_db",
        user="citus_mkt_qa",
        host="internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com",
        password="rn*Wh%osCl2C",
        port=7432
    )
)

#sql = "SELECT tablename FROM pg_tables"
#sql = "SELECT a.attnum,a.attname AS field,t.typname AS type,a.attlen AS length,a.atttypmod AS lengthvar,a.attnotnull AS notnull FROM pg_class c,pg_attribute a,pg_type t WHERE c.relname = 'market_adapp_pubapp_fact_v1' and a.attnum > 0 and a.attrelid = c.oid and a.atttypid = t.oid ORDER BY a.attnum;"
#sql = "select distinct(granularity) from market.market_adapp_publisher_fact_v1;"
#sql = "select * from market.market_adapp_pubapp_seen_fact_v1 where ad_app_id = 553834731 and country_code='JP' and device_code = 'ios-phone' limit 1;"
#sql = "select min(first_seen_date), max(first_seen_date) from market.market_creative_adapp_seen_fact_v1 where ad_app_id = 553834731 and country_code='US' and device_code = 'ios-phone' and ad_platform_id=116;"
#sql = "select count(*) from market.market_creative_adapp_seen_fact_v1 where last_seen_date is null;"
#sql = "select * from market.market_pubapp_cat_rank_fact_v1 where date='2019-11-02' and category_id=400035 and country_code='US' limit 1;"
#sql = "select distinct(category_id) from market.market_pubapp_cat_rank_fact_v1 where date='2019-11-02' and country_code='US'"
#sql = "select distinct(category_id) from market.market_adapp_cat_rank_fact_v1 where ad_app_id = 553834731 and country_code='US' and device_code = 'ios-phone' and granularity='weekly' and date='2019-10-19';"
#sql = "select * from market.market_pubapp_fact_v1 where pub_app_id=870099476 and country_code='US' and device_code = 'ios-phone' and granularity='weekly' and ad_platform_id=101;"
#sql = "select pub_app_count, pub_app_list from market.market_adapp_fact_v1 where ad_app_id=553834731 and country_code='US' and device_code = 'ios-phone' and granularity='weekly' and date='2019-11-02' and ad_platform_id=121;"
#rank_pub_of_ad
#sql = "select ad_app_id,pub_app_list from market.market_adapp_fact_v1 where pub_app_list is not null limit 10;"
# sov_ad_of_pub, sov_pub_of_ad, rank_ad_of_pub
#sql = "select count(pub_app_id) from market.market_adapp_pubapp_fact_v1 limit 1;"
#creative_count
#sql = "select ad_app_id from market.market_adapp_fact_v1 where creative_count is not null limit 50;"
#ad_of_publisher
#sql = 'select ad_app_id, publisher_id from market.market_adapp_publisher_fact_v1 limit 50;'
#creative_of_platform
#sql = "select creative_id from market.market_creative_adapp_fact_v1 limit 50;"
#pub_of_platform
#sql = "select pub_app_id from market.market_pubapp_fact_v1 where pub_app_share_of_ad_platform is not null limit 50;"
#sql = "select * from market.market_creative_log_fact_v1 where date = '2019-11-04' limit 1;"
#sql="select * from aso.aso_search_visibility_fact_v1 limit 5"
#ad_of_pub_seen
#sql = "select pub_app_id from market.market_adapp_pubapp_seen_fact_v1 where country_code = 'US' and first_seen_date is not null and last_seen_date is not null and ad_app_id=1020357030 limit 50"
#sql = "select count(*) from market.market_adapp_cat_rank_fact_v1;"
#sql = "select distinct(ad_platform_list) from market.market_adapp_pubapp_fact_v1 where ad_app_id=927060395 and ad_platform_count=1 limit 50;"
#sql = "select ad_platform_id, ad_app_id, ad_platform_count, ad_platform_list, date from market.market_adapp_fact_v1 where ad_app_id = 683970365 and device_code = 'ios-phone' and country_code = 'US' and granularity = 'weekly' and ad_platform_id !=110 limit 10;"
#sql = "select ad_platform_count, ad_platform_list from market.market_adapp_fact_v1 where ad_platform_count is not null and ad_platform_list is not null and ad_platform_id !=1 limit 10;"
#sql = "select ad_app_id, ad_platform_count, ad_platform_list, date from market.market_adapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null and ad_platform_id =1 limit 10;"
#sq1l = "select * from market.market_adapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null and ad_platform_id =1 limit 1;"
#sql2 = "select count(*) from market.market_adapp_fact_v1 where pub_app_count is null and pub_app_list is not null and ad_platform_id =1 limit 10;"
#sql2 = "select * from market.market_adapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null and ad_platform_id =1 and pub_app_count is not null and pub_app_list is not null and ad_platform_id =1 limit 10;"
#sql1 = "select count(*) from market.market_adapp_pubapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null limit 10;"
#sql2 = "select count(*) from market.market_adapp_pubapp_fact_v1 where ad_platform_count is not null and ad_platform_list is null limit 10;"
#sql3 = "select count(*) from market.market_adapp_pubapp_fact_v1;"
#sql1 = "select count(*) from market.market_creative_adapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null limit 10;"
#sql2 = "select count(*) from market.market_creative_adapp_fact_v1 where ad_platform_count is not null and ad_platform_list is null limit 10;"

#sql1 = "select * from market.market_creative_adapp_fact_v1 where pub_app_count is null and pub_app_list is not null and date = '2019-05-04' and granularity = 'weekly' limit 1;"
#sql2 = "select count(*) from market.market_creative_adapp_fact_v1 where pub_app_count is not null and pub_app_list is null limit 10;"

sql2 = "select * from market.market_creative_adapp_fact_v1 where device_code='android-tablet' and date = '2019-05-04' and granularity = 'weekly' and ad_app_id=20600006320009 and country_code = 'WW' and ad_platform_id=136  limit 10;"
#sql2 = "select date, granularity from market.market_creative_adapp_fact_v1 where country_count is not null and country_list is null and date > '2018-07-07' limit 10;"

#sql3 = "select count(*) from market.market_creative_adapp_fact_v1;"
#sql1 = "select count(*) from market.market_pubapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null and date > '2018-07-31' and granularity = 'monthly' limit 10;"
#sql2 = "select count(*) from market.market_pubapp_fact_v1 where ad_platform_count is not null and ad_platform_list is null limit 10;"
#sql3 = "select count(*) from market.market_pubapp_fact_v1;"
#sql = "select count(*) from market.market_pubapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null and ad_platform_id =1;"
#sql = "select count(*) from market.market_pubapp_fact_v1 where ad_platform_id =1;"
#sql1 = "select * from market.market_adapp_fact_v1 where ad_platform_count is null and ad_platform_list is not null and date > '2018-07-07' and granularity = 'weekly' limit 10;"
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
#test_result1 = query(citus_dsn_, sql1)
test_result2 = query(citus_dsn_, sql2)
#test_result3 = query(citus_dsn_, sql3)
#print 'device_code,','country_code:','granularity:', 'date:','ad_platform_id:','ad_app_id:', 'pub_app_id:','ad_platform_count:','ad_platform_list:','ad_app_share_of_pub_app:','pub_app_share_of_ad_app'
#print '\033[31mdevice_code:\033[0m', test_result1[0][0], '\033[31mcountry_code:\033[0m',test_result1[0][1], '\033[31mgranularity:\033[0m', test_result1[0][2], '\033[31mdate:\033[0m', test_result1[0][3],'\033[31mad_platform_id:\033[0m\033[0m',test_result1[0][4], '\033[31mad_app_id:\033[0m', test_result1[0][5], '\033[31mpub_app_id:\033[0m', test_result1[0][6], '\033[31mad_platform_count:\033[0m', test_result1[0][7], '\033[31mad_platform_list:\033[0m', test_result1[0][8], '\033[31mad_app_share_of_pub_app:\033[0m', test_result1[0][9], '\033[31mpub_app_share_of_ad_app:\033[0m', test_result1[0][10]
from pprint import pprint
pprint(test_result2)


In [0]:

import datetime


from aaintdatapipeline.application.app_qa.common.utils import is_in_current_month, datetime_to_string, \
    get_random_day_in_last_month
from aaintdatapipeline.core.loader import es
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.core.utils import email
from aaintdatapipeline.core.utils.spark import canned_spark, stop

urn = Urn(
    namespace="app-qa.db-check.v1",
)
common_config = {
    "database": "app_int"
}
es_conn = es.connection(urn, common_config)
granularity='monthly'
sdk_history_indexes = es_conn.indices.get("int_sdk_metric_history_*_*_{}_*".format(granularity)).keys()
params = {
    'conflicts': 'proceed',
    'wait_for_completion': False
}
query_body ={
  "query": {
    "term": {
      "date": {
        "value": "2018-10-12"
      }
    }
    
  }
}
result = es_conn.search(sdk_history_indexes, body=query_body)
print result

In [0]:
%%sh
curl -XGET http://internal-aa-int-sdk-elasticsearch-621265199.us-east-1.elb.amazonaws.com:19200/int_sdk*/_search?pretty

In [0]:


df = spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2019-05-04")
#df = df.filter("pub_app_ad_app_NETLST is not null and pub_app_ad_app_NETCNT is null")
df = df.filter("device_id=2 and network_id=136 and ad_app_id=20600006320009 and country_code = 'WW'")
df.show(10)


In [0]:


df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.adapp-fact.v1/fact/granularity=weekly/date=2019-05-04/device_code=android-tablet")
#df = df.filter("pub_app_ad_app_NETLST is not null and pub_app_ad_app_NETCNT is null")
df = df.filter("ad_app_id=20600006320009 and country_code = 'WW' and ad_platform_id=136")
df.show(10)



In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-market/unified/